In [4]:
import gym
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
import numpy as np

def initiate_NN(learning_rate=1e-3):
    model = Sequential()
    model.add(Dense(64, input_dim=8, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
    return model

def D_DQN(gamma,epsilon_decay,learning_rate,max_e,min_e,replay_batch_size=30,max_iterations=5000):
    env = gym.make('LunarLander-v2')
    state=env.reset()
    online_model = initiate_NN(learning_rate)
    target_model =initiate_NN(learning_rate)
    target_model.set_weights(online_model.get_weights())
    memory=[]
    episode_reward_rolling = []
    episode_reward=[]
    for episode in range(max_iterations):
        state=env.reset()
        current_episode_reward = 0
        done=False
        steps=0
        epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
        while not done:
            random_value = np.random.random()
            if random_value<=epsilon:
                action = env.action_space.sample()
            else:
                Y=online_model.predict(state.reshape(1,-1))
                action = np.argmax(Y[0])
            new_state, reward, done, info= env.step(action)
            steps+=1
            current_episode_reward +=reward
            memory.append([state,action,reward,new_state,done])
            state = new_state
            if len(memory)>=replay_batch_size:
                batch = random.sample(memory, replay_batch_size)
                batch = np.array(batch)
                non_terminal_samples = np.where(batch[:, 4] == False)
                y = np.copy(batch[:, 2])
                if len( non_terminal_samples[0]) > 0:
                    #use online q values to select best action
                    online_q = online_model.predict(np.vstack(batch[:, 3]))
                    #use offline q values of best action as target 
                    target_q = target_model.predict(np.vstack(batch[:, 3]))
                    y[non_terminal_samples] += np.multiply(gamma, \
                                target_q[non_terminal_samples, \
                                np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
                actions = np.array(batch[:, 1], dtype=int)
                y_target = online_model.predict(np.vstack(batch[:, 0]))
                y_target[range(replay_batch_size), actions] = y
                online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)
            if done:
                target_model.set_weights(online_model.get_weights())
        episode_reward.append(current_episode_reward) 
        if len(memory)>=5000:
            memory = memory[-5000:]
        if len(episode_reward)>=100:
            episode_reward_rolling.append(np.average(episode_reward[-100:]))
        else:
            episode_reward_rolling.append(np.average(episode_reward))
        print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
              ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
              '%.2f' % epsilon)
        if np.sum(np.array(episode_reward_rolling[-20:])>200)>19 and epsilon<0.1:
            break
    return online_model, episode_reward, episode_reward_rolling



if __name__=='__main__':
    gamma=0.99
    epsilon_decay = 0.01
    learning_rate =0.0005
    max_e=1
    min_e=0.0001
    replay_batch_size=30
    max_iterations=5000
    online_model, episode_reward, episode_reward_rolling = D_DQN(gamma,epsilon_decay,learning_rate,max_e,min_e,replay_batch_size=30,max_iterations=1)
    online_model.save_weights('weights/DQN_test_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'a.h5')
    import pickle
    with open('weights/DQN_test'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                             +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'a.json', 'wb') as fp:
        pickle.dump({'rewards':episode_reward,'rolling_reward':episode_reward_rolling}, fp)

episode:  0  episode reward:  -404.43  rolling reward:  -404.43  Steps:  116  epsilon:  1.00
